In [45]:
import os
import sys
import builtins
#os.environ['TRANSFORMERS_CACHE'] = '/project/SDS/research/christ_research/Llama 2/mammoth/cache'
os.environ['TRANSFORMERS_CACHE'] = '/scratch/brc4cb/llama/cache'
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch
from collections import defaultdict
import copy
import json
import os
from os.path import exists, join, isdir
from dataclasses import dataclass, field
import sys
from typing import Optional, Dict, Sequence
import numpy as np
from tqdm import tqdm
import logging
import bitsandbytes as bnb
import pandas as pd
import random

import torch
import transformers
from torch.nn.utils.rnn import pad_sequence
import argparse
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    set_seed,
    Seq2SeqTrainer,
    BitsAndBytesConfig,
    LlamaTokenizer

)
from datasets import load_dataset, Dataset
import evaluate

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)
from peft.tuners.lora import LoraLayer
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
#from adapter-transformers import AdapterType, AdapterConfig, load_adapter

# Set the environment variable
os.environ["HF_REMOTES_OFFLINE"] = "1"
#from dotenv import load_dotenv
# Load the environmental variables from the .env file
#load_dotenv()

#token= os.environ['huggingface_token']

#from huggingface_hub import login
#login(token = token)

# Redirect stdin to /dev/null
sys.stdin = open(os.devnull)

model_path = "meta-llama/Meta-Llama-3-8B"   # Specify the path to the model
#adapter_path = "mathwell8b-kto/checkpoint-1250"  # Specify the path to the adapter weights
adapter_path = "mathwell/llama3-8b-two_stage/checkpoint-250"  # Specify the path to the adapter weights
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="right") #, use_auth_token=True,)

# Patch the built-in input function to return 'y' automatically
def mock_input(prompt=None):
    return 'y'

# Patch the input function to use the mock_input function
builtins.input = mock_input

try:
    # Attempt to load the model with trust_remote_code=True
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        load_in_4bit=True, 
       # max_memory=max_memory,
        torch_dtype=torch.bfloat16,
        #use_auth_token=True,
        config=AutoConfig.from_pretrained(model_path, trust_remote_code=True)
    )
except EOFError:
    # If an EOFError occurs, provide the expected input ('y')
    pass

# Restore stdin
sys.stdin = sys.__stdin__

# Set special tokens
DEFAULT_PAD_TOKEN = "[PAD]"
def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=64)

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg
        
if tokenizer._pad_token is None:
    smart_tokenizer_and_embedding_resize(
        special_tokens_dict=dict(pad_token=DEFAULT_PAD_TOKEN),
        tokenizer=tokenizer,
        model=model,
    )

print('Adding special tokens.')
tokenizer.add_special_tokens({
        "eos_token": tokenizer.convert_ids_to_tokens(model.config.eos_token_id),
        "bos_token": tokenizer.convert_ids_to_tokens(model.config.bos_token_id),
#                 "unk_token": tokenizer.convert_ids_to_tokens(
#                     model.config.pad_token_id if model.config.pad_token_id != -1 else tokenizer.pad_token_id
#                 ),
})

# Load the adapter weights
model = PeftModel.from_pretrained(model, adapter_path)
import pandas as pd
import random

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Adding special tokens.


In [32]:
df = pd.read_csv('data/evaluation_annotations.csv')
df = df[df['good']==1]
df['output'] = "Question: " + df['question'] + "\n" + "Solution:\n" + df['solution']

In [33]:
addition = df.query("addition==1 and total_ops==1")
subtraction = df.query("subtraction==1 and total_ops==1")
multiplication = df.query("multiplication==1 and total_ops==1")
division = df.query("division==1 and total_ops==1")
fractions = df.query("fractions==1 and total_ops==1")
decimals = df.query("decimals==1 and total_ops==1")
multi_ops = df.query("total_ops>1")

In [34]:
print(len(addition), len(subtraction), len(multiplication), len(division), len(fractions), len(decimals), len(multi_ops))

69 79 123 78 0 0 528


In [62]:
topics = ['Superman', "Batman", "Wonder Woman", "Barbie", "Power Rangers", "basketball", "soccer", "football", "volleyball", 'field hockey',\
'Fortnite', 'Spiderman', "Iron Man", "Captain America", "Captain Marvel", "Thor, the God of Thunder", "Ninja Turtles", "Black Panther", "Taylor Swift", "swimming",\
"Pokémon", "Super Mario", "Naruto", "unicorns", "Hello Kitty", "Minecraft", "lacrosse", "cheer leading", "LeBron James", "Steph Curry", "Patrick Mahomes",\
"Serena Williams", "dogs", "cats", "dinosaurs", "Harry Potter", "cars", "planes", "trains", "pizza", "cookies", "ice cream", 'candy']
def prompt(df, model, tokenizer, n_qs, operation, operator, topics):
    responses = []
    while len(responses)<n_qs:
        topic = random.choice(topics)
        final_prompt = f"Write a grade school math {operation} word problem about {topic} and Python function with a commented out step-by-step solution to solve the word problem. The question you write should only require {operation} to solve, meaning the solution should rely only on use of the {operator} operator."
        prompt = f"Write a grade school math {operation} word problem and Python function with a commented out step-by-step solution to solve the word problem. The question you write should only require {operation} to solve, meaning the solution should rely only on use of the {operator} operator."
        #final_prompt = f"Write a grade school math {operation} word problem about {topic} and Python function with a commented out step-by-step solution to solve the word problem."
        #prompt = f"Write a grade school math {operation} word problem and Python function with a commented out step-by-step solution to solve the word problem."
        questions = []
        while len(questions)<8:
            question = df['output'].iloc[random.randint(0,len(df)-1)]
            if question not in questions:
                questions.append(question)
        formatted_prompt = []
        for i in range(0,8):
            formatted_prompt.append((f"Below is an instruction that describes a task. "
                    f"Write a response that appropriately completes the request.\n\n"
                    f"### Instruction:\n{prompt}\n\n### Response: {questions[i]}"))
        formatted_prompt.append(f"Below is an instruction that describes a task. "
                    f"Write a response that appropriately completes the request.\n\n"
                    f"### Instruction:\n{final_prompt}\n\n### Response: ")
        formatted_prompt = "\n".join(formatted_prompt)
        inputs = tokenizer.encode(formatted_prompt, return_tensors="pt")
        attention_mask = torch.ones_like(inputs)
        inputs = inputs.to('cuda')
        output = model.generate(inputs=inputs, attention_mask=attention_mask, max_new_tokens = 200, do_sample = True)
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Split the generated text by the prompt to extract the newly generated part
        generated_text_parts = generated_text.split(final_prompt)
        newly_generated_text = generated_text_parts[-1].strip()
        if "\nBel" in newly_generated_text:
            newly_generated_text = newly_generated_text.split("\nBel")[0]
        if "Question:" in newly_generated_text:
            responses.append(newly_generated_text)
    return responses
        # output_file = "mathwell8b_kto_questions_few.txt"  # Specify the path and filename for the output file
        # with open(output_file, "a") as f:  # Open the file in append mode ("a")
        #     f.write(f"Topic: {topic} " + newly_generated_text + "\n")  # Append the newly generated text to the file
    

In [ ]:
qs = prompt(addition, model, tokenizer, 30, 'addition', "+", topics=topics)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
q_len = len(qs)
add = 0
for i in range(0, len(qs)):
    try:
        solution = qs[i].split("Solution:")[1]
        if " - " not in solution and " * " not in solution and "/" not in solution and "." not in solution:
            add+=1
    except:
        q_len-=1
print(add/q_len)

In [ ]:
q_len = len(qs)
add = 0
for i in range(0, len(qs)):
    try:
        solution = qs[i].split("Solution:")[1]
        if " + " in solution:
            add+=1
    except:
        q_len-=1
print(add/q_len)

In [77]:
qs = prompt(subtraction, model, tokenizer, 30, 'subtraction', "-", topics=topics)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [78]:
q_len = len(qs)
sub = 0
for i in range(0, len(qs)):
    try:
        solution = qs[i].split("Solution:")[1]
        if " + " not in solution and " * " not in solution and "/" not in solution and "." not in solution:
            sub+=1
    except:
        q_len-=1
print(sub/q_len)

0.6551724137931034


In [79]:
q_len = len(qs)
sub = 0
for i in range(0, len(qs)):
    try:
        solution = qs[i].split("Solution:")[1]
        if " - " in solution:
            sub+=1
    except:
        q_len-=1
print(sub/q_len)

0.8620689655172413


In [74]:
qs = prompt(multiplication, model, tokenizer, 30, 'multiplication', "*", topics=topics)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [75]:
q_len = len(qs)
mult = 0
for i in range(0, len(qs)):
    try:
        solution = qs[i].split("Solution:")[1]
        if " + " not in solution and " - " not in solution and "/" not in solution and "." not in solution:
            mult+=1
    except:
        q_len-=1
print(mult/q_len)

0.7


In [76]:
q_len = len(qs)
mult = 0
for i in range(0, len(qs)):
    try:
        solution = qs[i].split("Solution:")[1]
        if "*" in solution:
            mult+=1
    except:
        q_len-=1
print(mult/q_len)

0.6666666666666666


In [67]:
qs = prompt(division, model, tokenizer, 30, 'division', "/", topics=topics)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [69]:
q_len = len(qs)
div = 0
for i in range(0, len(qs)):
    try:
        solution = qs[i].split("Solution:")[1]
        if " + " not in solution and " - " not in solution and " * " not in solution and "." not in solution:
            div+=1
    except:
        q_len-=1
print(div/q_len)

0.21428571428571427


In [72]:
q_len = len(qs)
div = 0
for i in range(0, len(qs)):
    try:
        solution = qs[i].split("Solution:")[1]
        if "/" in solution:
            div+=1
    except:
        q_len-=1
print(div/q_len)

0.32142857142857145


In [ ]:
(0.21428571428571427+0.6206896551724138+0.75+0.4827586206896552)/4